
# Seed → Augmented Dataset (Jupyter)

**뼈대(Seed) CSV**를 불러와서, **의미 보존 의역 / 암시적 의도 / 메신저 노이즈**를 섞은 **증강 데이터**를 생성합니다.  
출력 컬럼: `sentence, domain, task, label, confidence, source, seed_id`

> 실행 전에 `OPENAI_API_KEY`(필수)와 필요 시 `OPENAI_BASE_URL`(팀 전용 게이트웨이)을 설정하세요.


In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## 0) 필요시 설치

In [2]:

# 필요시 한 번 실행
# %pip install openai pandas tqdm tenacity python-dateutil
# (선택) 유사도 기반 중복제거를 쓰려면 아래 설치
# %pip install scikit-learn
# %pip install torch torchvision
# %pip install transformers peft accelerate bitsandbytes datasets -q -U
# %pip install pyngrok
# %pip install jupyter-server-proxy

## 1) 설정

In [2]:

import os

# OpenAI API를 활용 시
# === 필수: OPENAI_API_KEY ===
# os.environ["OPENAI_API_KEY"] = "S13P32S101-f2a0e414-d636-44a5-982a-8230a5edb0db"

# === 선택: 팀 전용 게이트웨이 ===
# 예: "https://your-team-endpoint.example.com/v1"
# os.environ["OPENAI_BASE_URL"] = "https://gms.ssafy.io/gmsapi/api.openai.com/v1"

# os.environ.setdefault("OPENAI_MODEL", "gpt-4o")

# OpenSource LLM 모델 활용 시
os.environ["BASE_MODEL_ID"] = "Qwen/Qwen3-4B-Instruct-2507"

# === 입출력 경로 ===
os.environ.setdefault("SEED_CSV", "schedule_dataset.csv")   # seed 파일 경로
os.environ.setdefault("OUTPUT_CSV", "schedule_dataset_augmented_epoch_2.csv")

# === 생성/증강 파라미터 ===
os.environ.setdefault("DOMAIN", "design_production")

# seed 한 줄당 몇 개 생성할지
os.environ.setdefault("AUG_PER_SEED", "1")

# teacher 확신도 하한
os.environ.setdefault("CONFIDENCE_MIN", "0.75")

# 일정(Task ∈ CREATE/UPDATE/CANCEL/INFO) 중 암시적 의도 비율
os.environ.setdefault("IMPLICIT_INTENT_RATIO", "0.35")

# 메신저 노이즈(인용/멘션/첨부/이모지) 삽입 비율 (0~1)
os.environ.setdefault("NOISE_RATIO", "0.1")

# 배치 단위 (한 번에 몇 개의 항목을 생성시킬지)
os.environ.setdefault("BATCH_SIZE", "10")

# 생성 다양성
os.environ.setdefault("TEMPERATURE", "0.8")
os.environ.setdefault("TOP_P", "0.9")

print("환경설정 완료")


환경설정 완료


## 2) 라이브러리 로드

In [9]:

import json, csv, random, time
from typing import List, Dict, Any

import pandas as pd
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential

# OpenAI SDK (신/구버전 호환)
from openai import OpenAI
NEW_SDK = True

SEED_CSV = os.getenv("SEED_CSV")
OUTPUT_CSV = os.getenv("OUTPUT_CSV")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4.1")
BASE_MODEL_ID = os.getenv("BASE_MODEL_ID", "Qwen/Qwen3-VL-4B-Instruct")

DOMAIN = os.getenv("DOMAIN", "design_production")
AUG_PER_SEED = int(os.getenv("AUG_PER_SEED", "1"))
CONFIDENCE_MIN = float(os.getenv("CONFIDENCE_MIN", "0.75"))
IMPLICIT_INTENT_RATIO = float(os.getenv("IMPLICIT_INTENT_RATIO", "0.35"))
NOISE_RATIO = float(os.getenv("NOISE_RATIO", "0.1"))
BATCH_SIZE = int(os.getenv("BATCH_SIZE", "10"))
TEMPERATURE = float(os.getenv("TEMPERATURE", "0.8"))
TOP_P = float(os.getenv("TOP_P", "0.9"))
EPOCH = int(os.getenv("EPOCH", "3"))

random.seed(7)

print("AUG_PER_SEED", AUG_PER_SEED)
print("EPOCH", EPOCH)
print("SEED_CSV:", SEED_CSV)
print("OUTPUT_CSV:", OUTPUT_CSV)
print("MODEL:", OPENAI_MODEL)
print("BASE_URL:", OPENAI_BASE_URL)
print("BASE_MODEL_ID:", BASE_MODEL_ID)


AUG_PER_SEED 1
EPOCH 3
SEED_CSV: schedule_dataset.csv
OUTPUT_CSV: schedule_dataset_augmented_epoch_2.csv
MODEL: gpt-4.1
BASE_URL: https://api.openai.com/v1
BASE_MODEL_ID: Qwen/Qwen3-4B-Instruct-2507


## 3) Seed 데이터 불러오기

In [10]:

df_seed = pd.read_csv(SEED_CSV)
# 예상 컬럼: sentence, domain, task, label, confidence, source
# 최소 필요: sentence, task, label
missing = [c for c in ["sentence","task","label"] if c not in df_seed.columns]
if missing:
    raise ValueError(f"Seed CSV에 필요한 컬럼이 없습니다: {missing}")

df_seed["domain"] = df_seed.get("domain", DOMAIN)
df_seed = df_seed.reset_index().rename(columns={"index":"seed_id"})
print(df_seed.head(3))
print("Seed rows:", len(df_seed))


   seed_id             sentence             domain    task  label  confidence  \
0        0     배경 러프 수정본 여기 있어요  design_production    NONE      0        0.99   
1        1  오늘 밤에 색감 체크 가능하신가요?  design_production  CREATE      1        0.92   
2        2        싱크표 다시 올려드릴게요  design_production    NONE      0        0.98   

    source  
0  teacher  
1  teacher  
2  teacher  
Seed rows: 500


## 4) 프롬프트 (증강 규칙)

In [11]:

SYSTEM_PROMPT = f"""You are a data generator for Korean chat augmentation in a single domain: "{DOMAIN}".
Your job: For each input item (seed sentence with its task/label), produce short, natural **augmented** Korean chat messages
that preserve the original task/label semantics.

Tasks: CREATE, UPDATE, CANCEL, INFO, NONE.
- If NONE: the output must not imply any scheduling intent.
- Otherwise (CREATE/UPDATE/CANCEL/INFO): the output **must** clearly or implicitly imply scheduling intent that matches the task.

Augmentation Styles:
1) Paraphrase while **preserving meaning** (task/label invariant).
2) Implicit intent (no obvious keywords like '일정/회의/취소' but intent must be clear) for roughly IMPLICIT_INTENT_RATIO of non-NONE.
3) Messenger-style noise (quote lines, @mentions, [attachments], emojis) for roughly NOISE_RATIO overall; keep label unchanged.

Constraints:
- Keep outputs short (1~20 tokens-ish), mix 존댓말/반말, allow slang/emojis, Korean only.
- Include time expressions naturally when helpful (내일/금요일/14:00/30분 등).
- No personal data (no real names/phones/URLs).

Output strictly in JSON (UTF-8, no comments) with schema per item:
{{
  "sentence": "<augmented sentence>",
  "task": "CREATE|UPDATE|CANCEL|INFO|NONE",
  "label": 0 or 1,
  "confidence": 0.0-1.0,
  "rationale": "<short Korean reason>"
}}
"""

def build_user_prompt(batch_items: List[Dict[str, Any]]) -> str:
    # batch_items: list of {"sentence","task","label"} from seed
    items_json = json.dumps(batch_items, ensure_ascii=False)
    return f"""다음 seed 항목들을 의미 보존으로 증강해 주세요.
- 각 seed에 대해 {int(os.getenv("AUG_PER_SEED","3"))}개씩 생성하세요.
- NONE은 일정 의도가 없도록, 나머지는 해당 task의 의도를 유지하세요.
- 일정 샘플의 약 {float(os.getenv("IMPLICIT_INTENT_RATIO","0.35")):.0%}는 '의도(암시)' 표현으로 만드세요.
- 메신저 노이즈 약 {float(os.getenv("NOISE_RATIO","0.3")):.0%} 적용(인용, 멘션, 첨부, 이모지).
- 중복/근사중복 피하기.

입력 seed 목록(JSON):
{items_json}

출력 형식(반드시 이 형태의 JSON):
{{ "items": [ {{...}}, {{...}} ] }}
"""


## 5-1) OpenAI 호출 유틸

In [ ]:
from typing import Optional

def get_client():
    if not OPENAI_API_KEY:
        raise RuntimeError("OPENAI_API_KEY not set.")
    if NEW_SDK:
        return OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
    else:
        openai.api_key = OPENAI_API_KEY
        openai.api_base = OPENAI_BASE_URL
        return None

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def call_openai(system_prompt: str, user_prompt: str) -> str:
    if NEW_SDK:
        client = get_client()
        resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role":"system","content":system_prompt},
                {"role":"user","content":user_prompt},
            ],
            temperature=float(os.getenv("TEMPERATURE","0.8")),
            top_p=float(os.getenv("TOP_P","0.9")),
            response_format={"type":"json_object"},
        )
        return resp.choices[0].message.content
    else:
        resp = openai.ChatCompletion.create(
            model=OPENAI_MODEL,
            messages=[
                {"role":"system","content":system_prompt},
                {"role":"user","content":user_prompt},
            ],
            temperature=float(os.getenv("TEMPERATURE","0.8")),
            top_p=float(os.getenv("TOP_P","0.9")),
        )
        return resp["choices"][0]["message"]["content"]

def safe_parse_json(s: str) -> Dict[str, Any]:
    s = s.strip()
    if "{" in s and "}" in s:
        s = s[s.find("{"): s.rfind("}")+1]
    try:
        return json.loads(s)
    except Exception:
        s = s.replace("\n"," ").replace("\t"," ")
        try:
            return json.loads(s)
        except Exception:
            return {}


## 5-2) Opensource LLM 호출 유틸

In [6]:
from typing import Optional
import torch
from transformers import pipeline

pipe = None  # 전역 변수

def get_LLM():
    global pipe
    if pipe is None:
        if not BASE_MODEL_ID:
            raise RuntimeError("BASE_MODEL_ID not set.")
        pipe = pipeline("text-generation", model=BASE_MODEL_ID)
    return pipe

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def begin_pipeline(system_prompt: str, user_prompt: str) -> str:
    pipe = get_LLM()
    if pipe:
        try:
            messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]
            outputs = pipe(messages, max_new_tokens=2048)
            return outputs[0].get('generated_text')[2]["content"]
        except Exception:
            pass

import json, re
from typing import Any, Dict, List

def safe_parse_json(payload: Any) -> Dict[str, Any]:
    """
    Robust JSON extractor:
    - payload가 dict/list/str 어떤 형태여도 처리
    - assistant.content만 추출
    - 바깥 { ... } 슬라이스
    - 흔한 깨짐 복구: 개행/탭/작은따옴표/트레일링 콤마
    """
    # 1) assistant 텍스트만 뽑기
    text = ""

    # 케이스 A: {'role':'assistant','content':'{...}'}
    if isinstance(payload, dict) and "content" in payload:
        text = str(payload.get("content", ""))

    # 케이스 B: [{'generated_text': '...'}] 또는 [{'generated_text': [{'role':..., 'content': '...'}]}]
    elif isinstance(payload, list) and payload:
        first = payload[0]
        gen = first.get("generated_text") if isinstance(first, dict) else None
        if isinstance(gen, str):
            text = gen
        elif isinstance(gen, list):
            # role 리스트라면 뒤에서부터 assistant content 찾기
            for msg in reversed(gen):
                if isinstance(msg, dict) and msg.get("role") == "assistant":
                    text = msg.get("content", "")
                    break
            if not text:
                for msg in gen:
                    if isinstance(msg, dict) and "content" in msg:
                        text = msg["content"]
                        break
        else:
            # list인데 포맷이 다르면 문자열로 캐스팅
            text = str(first)

    # 케이스 C: 이미 문자열
    elif isinstance(payload, str):
        text = payload
    else:
        text = str(payload or "")

    s = text.strip()
    if not s:
        return {}

    # 2) 바깥쪽 { ... }만 슬라이스
    if "{" in s and "}" in s:
        s = s[s.find("{"): s.rfind("}") + 1]

    # 3) 1차 파싱 시도
    try:
        return json.loads(s)
    except Exception:
        pass

    # 4) 복구: 개행/탭 제거, 작은따옴표 -> 큰따옴표
    s_fix = s.replace("\n", " ").replace("\t", " ")
    # JSON 내 한국어/문장부호엔 영향 적으니 전체 치환 허용
    s_fix = s_fix.replace("'", '"')

    # 5) 복구: 트레일링 콤마 제거 (", }", ", ]")
    s_fix = re.sub(r",\s*([}\]])", r"\1", s_fix)

    # 6) 최종 파싱 시도
    try:
        return json.loads(s_fix)
    except Exception:
        return {}


In [15]:
from pprint import pprint
user_prompt = build_user_prompt(req_items)
raw = begin_pipeline(SYSTEM_PROMPT, user_prompt)
pprint(raw)
data = safe_parse_json(raw)
print("-=--------------------------------------------------------------")
print(data)
items = data.get("items", [])
print("-=--------------------------------------------------------------")
print(items)

NameError: name 'req_items' is not defined

## 6) 중복 제거 유틸 (정확/유사)

In [16]:

from typing import List, Dict, Any

def dedup_exact(items: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    seen = set()
    uniq = []
    for it in items:
        key = it["sentence"].strip()
        if key not in seen:
            uniq.append(it)
            seen.add(key)
    return uniq

# (선택) 코사인 유사도 기반 근사중복 제거
USE_COSINE = False
try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    USE_COSINE = True
except Exception:
    pass

def dedup_cosine(items: List[Dict[str, Any]], thr: float = 0.92) -> List[Dict[str, Any]]:
    if not USE_COSINE or len(items) < 2:
        return items
    texts = [x["sentence"] for x in items]
    vec = TfidfVectorizer(ngram_range=(1,2), min_df=1).fit_transform(texts)
    sim = cosine_similarity(vec)
    keep = []
    removed = set()
    for i in range(len(items)):
        if i in removed:
            continue
        keep.append(items[i])
        for j in range(i+1, len(items)):
            if sim[i, j] >= thr:
                removed.add(j)
    return keep


## 7) 증강 실행

In [18]:
import numpy as np

BASE_SEED = 7  # 예시: 현재 사용 중인 시드

def set_epoch_seed(seed_value: int):
    random.seed(seed_value)
    np.random.seed(seed_value)
    # 파이썬 해시 시드까지 고정하고 싶다면 (프로세스 시작 시 적용) :
    # os.environ["PYTHONHASHSEED"] = str(seed_value)

all_items: list[dict] = []

# 0-에폭(원본)도 포함
for _, row in df_seed.iterrows():
    all_items.append({
        "sentence": row["sentence"],
        "domain": row.get("domain", DOMAIN),
        "task": row["task"],
        "label": int(row["label"]),
        "confidence": float(row.get("confidence", 1.0)),
        "source": row.get("source", "seed"),
        "seed_id": int(row["seed_id"]),
        "epoch": 0,  # 원본 표시
    })

# 다음 에폭의 입력이 되는 현재 데이터프레임
# (다음 라운드 프롬프트 입력에 필요한 최소 컬럼 유지)
df_current = df_seed[["sentence", "task", "label"]].copy()

batch_size = max(1, int(BATCH_SIZE))

for ep in range(1, EPOCH + 1):
    # 에폭별 시드: 7, 17, 27 ... (BASE_SEED + 10*(ep-1))
    seed_for_ep = BASE_SEED + 10 * (ep - 1)
    set_epoch_seed(seed_for_ep)

    epoch_generated = []  # 이 에폭에서 새로 생성된 것들만 잠시 보관

    for start in tqdm(range(0, len(df_current), batch_size), desc=f"Augmenting e{ep}"):
        batch_df = df_current.iloc[start:start + batch_size]

        req_items = [{"sentence": r["sentence"], "task": r["task"], "label": int(r["label"])}
                     for _, r in batch_df.iterrows()]

        user_prompt = build_user_prompt(req_items)
        raw = begin_pipeline(SYSTEM_PROMPT, user_prompt)
        data = safe_parse_json(raw)
        items = data.get("items", [])

        normed = []
        for r in items:
            try:
                sent = str(r.get("sentence", "")).strip()
                task = str(r.get("task", "")).strip().upper()
                label = int(r.get("label", 1 if task != "NONE" else 0))
                conf = float(r.get("confidence", 0.0))
            except Exception:
                continue

            if not sent or task not in ["CREATE", "UPDATE", "CANCEL", "INFO", "NONE"]:
                continue
            if conf < CONFIDENCE_MIN:
                continue

            normed.append({
                "sentence": sent,
                "domain": DOMAIN,
                "task": task,
                "label": label,
                "confidence": conf,
                "source": "augmented",
                "seed_id": None,
                "epoch": ep,  # 어느 에폭에서 생성됐는지 기록
            })

        # 배치 내 중복 제거
        normed = dedup_exact(normed)
        normed = dedup_cosine(normed, thr=0.92)

        # 전체 결과에 합치고, 이번 에폭의 입력 후보에도 적재
        all_items.extend(normed)
        epoch_generated.extend(normed)

        time.sleep(0.2)

    # 다음 에폭을 위해, 이번 에폭에서 생성된 것만을 입력으로 사용
    if len(epoch_generated) == 0:
        # 더 이상 진화할 데이터가 없으면 조기 종료
        break

    df_current = pd.DataFrame(epoch_generated, columns=[
        "sentence", "task", "label", "confidence", "domain", "source", "seed_id", "epoch"
    ])

    # 다음 라운드 프롬프트 입력에 필요한 최소 컬럼만 유지
    df_current = df_current[["sentence", "task", "label"]].copy()

# 최종 CSV 저장 (원본 + 각 에폭 산출물 모두 포함)
df_aug = pd.DataFrame(all_items, columns=[
    "sentence", "domain", "task", "label", "confidence", "source", "seed_id", "epoch"
])
df_aug.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print("Saved:", OUTPUT_CSV)
print("Rows:", len(df_aug))
display(df_aug.head(10))

Augmenting e1:   0%|          | 0/50 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Augmenting e3: 100%|██████████| 50/50 [14:38<00:00, 17.56s/it]

Saved: schedule_dataset_augmented_epoch_2.csv
Rows: 2000


,sentence,domain,task,label,confidence,source,seed_id,epoch
0,배경 러프 수정본 여기 있어요,design_production,NONE,0,0.99,teacher,0.0,0
1,오늘 밤에 색감 체크 가능하신가요?,design_production,CREATE,1,0.92,teacher,1.0,0
2,싱크표 다시 올려드릴게요,design_production,NONE,0,0.98,teacher,2.0,0
3,금요일 오후 킥오프 미팅 할까요?,design_production,CREATE,1,0.95,teacher,3.0,0
4,오늘 콘티 피드백만 주세요,design_production,NONE,0,0.97,teacher,4.0,0
5,채색 일정 이번 주로 당길 수 있을까요?,design_production,UPDATE,1,0.89,teacher,5.0,0
6,배경 담당 누구로 할지 정해졌나요?,design_production,NONE,0,0.96,teacher,6.0,0
7,잉킹 리뷰 내일 오전이면 괜찮으실까요?,design_production,CREATE,1,0.91,teacher,7.0,0
8,오늘 회의 그냥 패스하죠,design_production,CANCEL,1,0.94,teacher,8.0,0
9,PSD 파일에 레이어 정리해놨습니다,design_production,NONE,0,0.99,teacher,9.0,0



## 8) 팁
- **라벨 불변**이 핵심입니다. UPDATE↔CANCEL 혼동은 `CONFIDENCE_MIN`을 올려 걸러내세요.
- 유사도 기반 중복제거를 쓰려면 `scikit-learn` 설치 후 자동 활성화됩니다.
- 증강 강도: `AUG_PER_SEED`, `TEMPERATURE`, `NOISE_RATIO`, `IMPLICIT_INTENT_RATIO`로 조절.
- 팀 전용 게이트웨이: `OPENAI_BASE_URL` 환경변수를 지정하세요.
